In [1]:
import numpy as np
import scipy as sp
import pandas as pd
from shapely.geometry import Polygon
from scipy.spatial.distance import directed_hausdorff

In [2]:
original = pd.read_csv('/root/autodl-tmp/location-privacy/data/test_latlon.csv')
syn_demo = pd.read_csv('/root/autodl-tmp/location-privacy/results/syn_traj_test.csv')

In [4]:
# IoU based on Convex Hull

syn_demo_IOU = []
# Hausdorff

syn_demo_Haus = []

tidlist = list(original['tid'].unique())

for tid in tidlist:
    real_traj = np.concatenate([np.array(original.loc[original['tid'] == tid]["lat"]).reshape(-1,1),
                                np.array(original.loc[original['tid'] == tid]["lon"]).reshape(-1,1)],
                               axis=1)
    syn_demo_traj = np.concatenate([np.array(syn_demo.loc[syn_demo['tid'] == tid]["lat"]).reshape(-1,1),
                               np.array(syn_demo.loc[syn_demo['tid'] == tid]["lon"]).reshape(-1,1)],
                              axis=1)
    
    syn_demo_Haus.append(directed_hausdorff(real_traj,syn_demo_traj)[0])
    
    real_traj_convex_hull = Polygon(real_traj).convex_hull
    syn_demo_traj_convex_hull = Polygon(syn_demo_traj).convex_hull
    syn_demo_I = real_traj_convex_hull.intersection(syn_demo_traj_convex_hull).area
    syn_demo_U = real_traj_convex_hull.union(syn_demo_traj_convex_hull).area
    syn_demo_IOU.append(syn_demo_I/syn_demo_U)

In [6]:
df = pd.DataFrame({
                   'syn_demo_IOU':syn_demo_IOU,
                   'syn_demo_Haus':syn_demo_Haus})
df.describe()

,syn_demo_IOU,syn_demo_Haus
count,1027.000000,1027.000000
mean,0.009568,0.239706
std,0.030599,0.122843
min,0.000000,0.026415
25%,0.000000,0.137941
50%,0.000000,0.238914
75%,0.000000,0.328614
max,0.274874,0.669555
